In [13]:
from google.colab import drive
import os.path
if not os.path.isdir("/content/drive"):
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#! wget -N https://raw.githubusercontent.com/kamawanu/zenn.dev-kamawanu-codes/main/htmlparser/htmlminidom.py

In [ ]:
#! wget -N https://raw.githubusercontent.com/kamawanu/zenn.dev-kamawanu-codes/main/mhtmlparser/mhtmlmini.py

In [14]:
import sys
import htmlminidom
import mhtmlmini
import importlib

In [15]:
import glob
import os.path
import os
import urllib
import time

In [16]:
def bepixiv(*,html,mhtml):
  ptitle = html.find("h1")
  if not ptitle is None:
    try:
      atitle = html.find("h2").find("a").find ("div").attrs["title"]
    except AttributeError as exc:
      print(exc,fn)
      return None
    ktags = []
    mtags = []
    for tagtag in html.find("footer").find("ul").finditer("li"):
        tag = tagtag[0][0][0]
        if type(tag) is str:
            ktags.append(f"#{tag}")
        if type(tag).__name__ == "dom" and tag.tag == "a":
            mtags.append(tag.content[0])
    ktags = " ".join(ktags)
    newfnn = f"pixiv {atitle} {ptitle[0]} {ktags} @{pu}.mhtml"
  else:
    newfnn = f"pixiv {roottitle} @{pu}.mhtml"
  return newfnn

In [17]:
def viewtitle(*,html,mhtml):
  print(mhtml.homeurl,html.find("title")[0].replace("\n",""))

In [18]:
def lastunique(*,html,mhtml):
  t = html.find("title")[0]
  u = urllib.parse.urlparse(mhtml.homeurl)
  p = u.path.split("/")
  t = t.replace(f"#{u.hostname}","").replace(f"#{p[-1]}","")
  r = f"{u.hostname} {p[-1]} {t}".replace(".html","").replace(".","_") + ".mhtml"
  return r

In [19]:
def deviantart(*,html,mhtml):
  t = html.find("title")[0]
  u = urllib.parse.urlparse(mhtml.homeurl)
  p = u.path.split("/")[1:]
  r = f"{u.hostname} {p[0]} {t}".replace(".","_") + ".mhtml"
  return r

In [20]:
def hostonly(*,html,mhtml):
  t = html.find("title")[0]
  u = urllib.parse.urlparse(mhtml.homeurl)
  r = f"{u.hostname} {t}".replace(".html","").replace(".","_") + ".mhtml"
  return r

In [21]:
methods = {
    "www.pixiv.net": bepixiv,
    #"iwamkumo.fanbox.cc": False,
    "www.rubbermagic.de": lastunique,
    "libidex.com": lastunique,
    "ja.aliexpress.com": lastunique,
    "www.deviantart.com": deviantart,
    "www.hwdesignshop.com": hostonly,
    "www.ebay.com": lastunique,
    "www.etsy.com": lastunique,
    "store.fetishfactory.com": lastunique,
    "www.wish.com": hostonly,
    "www.ebay.co.uk": lastunique,
    "www.dreammask.net": hostonly,
    "www.demask.com": hostonly,
    "www.simon-o.com": hostonly,
    "rubear.moscow": lastunique,
    "www.kinkprojects.com": hostonly,
    "www.desertcart.is": hostonly,
    'twitter.com': None,
    'jp.quora.com': hostonly, 
    'www.flickr.com': None, 
    'fetcherx.com': hostonly, 
    'www.eurocatsuits.com': hostonly, 
    'www.facebook.com': lastunique, 
    'm.facebook.com' : lastunique,
    'stock.adobe.com': hostonly,
    'www.one-tab.com': lastunique,
    "twitter.com": lastunique,
    'i.imgur.com': lastunique,
    'www.flickr.com': lastunique,
}

In [22]:
def pixivfanbox(*,html,mhtml):
  ll = tuple(html.finditer("h1"))
  aname = ll[0][0].content[0]
  xtitle = ll[-1].content[0]
  uz = urllib.parse.urlparse(mhtml.homeurl)
  u = uz.path.split("/")[-1]
  r = f"{uz.hostname} {aname} {xtitle} @{u}".replace(".","_")
  return r

In [23]:
unknown = set()
for fn in glob.glob("/content/drive/MyDrive/mhtml-incoming/*.mht*"):
  if ".DUP." in fn:
    os.remove(fn)
    continue
  assert os.path.isfile(fn)
  fndir = os.path.dirname(fn)
  fnname = os.path.basename(fn)

  dsdir = fndir.replace("incoming","named")
  if not os.path.isdir(dsdir):
    os.mkdir(dsdir)

  mhtml = False
  try:
    mhtml = mhtmlmini.from_file(fn)
    html = htmlminidom.htmlminiparser()
    behtml = mhtml.gethome().get_payload()
  #except TypeError as exc:
  #  print(f"?? {exc}",fn)
  #  continue
  except UnicodeDecodeError as exc:
    print(fn,exc,mhtml)
    continue
  assert type(behtml) is str, behtml
  html.feed(behtml)
  upz = urllib.parse.urlparse(mhtml.homeurl)
  pu = upz.path.split("/")[-1]
  newfnn = None
  roottitle = html.find("title")[0]

  _method = methods.get(upz.hostname)
  if _method is None and ( "latex" in upz.hostname ) or ( "rubber" in upz.hostname ):
    _method = hostonly
  if _method is None and ( "fanbox.cc" in upz.hostname ):
    _method = pixivfanbox

  if _method is False:
    print(fn,upz.hostname)
  elif _method is None:
    viewtitle(html=html,mhtml=mhtml)
    unknown.add(upz.hostname)
  else:
    newfnn = _method(html=html,mhtml=mhtml)
    #print(fn,mhtml.homeurl,html.find("title").content[0])
  if newfnn is not None:
    assert not "{" in newfnn
    if not ".mhtml" in newfnn:
      newfnn = newfnn + ".mhtml"
    newfnn = newfnn.replace("/","_").replace(".html","")
    #if fnname != newfnn:
    newfnn = newfnn.replace(f"##{upz.hostname}","")
    nande=(fn, f"{dsdir}/{newfnn}" )
    if os.path.isfile(nande[1]):
      if os.path.getmtime(nande[0]) < os.path.getmtime(nande[1]):
        os.remove(nande[0])
        assert not os.path.isfile(nande[0])
        continue
      else:
        os.remove(nande[1])
    assert os.path.isfile(nande[0]) and not os.path.isfile(nande[1]), nande
    try:
      os.rename(*nande)
    except OSError as exc:
      print(exc)
      continue
    assert os.path.isfile(nande[1]), f"rename failed {nande}"
    assert not os.path.isfile(nande[0]), f"rename failed {nande}"
    print(nande)
  pass#break
print(unknown)

TypeError: ignored

In [ ]:
import importlib
importlib.reload(mhtmlmini)